In [1]:
from pathlib import Path
import numpy as np
import joblib
import tensorflow
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.applications import vgg16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.applications.vgg16 import VGG16

In [2]:
#path to folders for training data
mask_path= Path("Face Mask Dataset") / "Train" / "WithMask"
no_mask_path= Path("Face Mask Dataset") / "Train" / "WithoutMask"


In [3]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 5
BS = 32

In [4]:
images = []
labels = []

In [5]:
for img in no_mask_path.glob("*.png"):
    #Load the image from disk
    img = image.load_img(img)
    
    img = img.resize([224,224])
    
    #Convert the image to an array
    image_array = image.img_to_array(img)
    
    #Add the image to the list of images
    images.append(image_array)
    
    #For each "not mask image", the expected value should be 0
    labels.append(0)

In [6]:
for img in mask_path.glob("*.png"):
    #Load the image from disk
    img = image.load_img(img)
    
    img = img.resize([224,224])
    
    #Convert the image to an array
    image_array = image.img_to_array(img)
    
    #Add the image to the list of images
    images.append(image_array)
    
    #For each "mask image", the expected value should be 1
    labels.append(1)

In [7]:
# Convert the array with images we loaded into a numpy array
images = np.array(images, dtype="float32")

# Convert the lables array into a numpy array
labels = np.array(labels)

In [8]:
# Normalize image data for train between 0 and 1
images = images/255

# Convert labels to categorical
labels = to_categorical(labels)


In [9]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(images, labels,test_size=0.2, stratify=labels, random_state=42)

In [10]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [11]:
# load the vgg16 network, ensuring the head FC layer sets are
# left off
baseModel = VGG16(weights="imagenet",input_shape=(224,224,3), include_top=False)

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    epochs=EPOCHS)

# # make predictions on the testing set
# print("[INFO] evaluating network...")
# predIdxs = model.predict(testX, batch_size=BS)

# # for each image in the testing set we need to find the index of the
# # label with corresponding largest predicted probability
# predIdxs = np.argmax(predIdxs, axis=1)

# # show a nicely formatted classification report
# print(classification_report(testY.argmax(axis=1), predIdxs,
#     target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("VGGMaskDetector.h5")

[INFO] compiling model...
[INFO] training head...


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5
250/250 [==============================] - 1611s 6s/step - loss: 0.6203 - accuracy: 0.6939 - val_loss: 0.4333 - val_accuracy: 0.8885
Epoch 2/5
250/250 [==============================] - 1589s 6s/step - loss: 0.4292 - accuracy: 0.8679 - val_loss: 0.2909 - val_accuracy: 0.9200
Epoch 3/5
250/250 [==============================] - 1566s 6s/step - loss: 0.3216 - accuracy: 0.9078 - val_loss: 0.2172 - val_accuracy: 0.9335
Epoch 4/5
250/250 [==============================] - 1566s 6s/step - loss: 0.2636 - accuracy: 0.9184 - val_loss: 0.1686 - val_accuracy: 0.9485
Epoch 5/5
250/250 [==============================] - 1575s 6s/step - loss: 0.2285 - accuracy: 0.9245 - val_loss: 0.1541 - val_accuracy: 0.9435
[INFO] saving mask detector model...
